In [28]:
import os

os.getcwd()

'c:\\Users\\YJKIM_PC\\pytorch_study\\sesac'

In [18]:
import os

dataPath = 'c:\\Users\\YJKIM_PC\\pytorch_study\\sesac\\Vehicles'
dataFolder = os.listdir(dataPath)
print(dataFolder)
print(len(dataFolder))

['Auto Rickshaws', 'Bikes', 'Cars', 'Motorcycles', 'Planes', 'Ships', 'Trains']
7


In [5]:
Vehicles_tvt = 'Vehicles_tvt'
v_train = os.path.join(Vehicles_tvt, 'train')
v_valid = os.path.join(Vehicles_tvt, 'valid')
v_test = os.path.join(Vehicles_tvt, 'test')

os.makedirs(v_train, exist_ok=True)
os.makedirs(v_valid, exist_ok=True)
os.makedirs(v_test, exist_ok=True)

for folder in dataFolder:
    train_temp = os.path.join(v_train, folder)
    os.makedirs(train_temp, exist_ok=True)
    valid_temp = os.path.join(v_valid, folder)
    os.makedirs(valid_temp, exist_ok=True)
    test_temp = os.path.join(v_test, folder)
    os.makedirs(test_temp, exist_ok=True)

In [11]:
# shutil move를 써서 지금은 0으로 나옴

for folder in dataFolder:
    temp0 = os.listdir(os.path.join(dataPath, folder))
    temp = len(temp0)
    print(temp)

0
0
0
0
0
0
0


In [7]:
import random, shutil

random.seed(0)

train_ratio = 0.8
valid_ratio = 0.15
test_ratio = 0.15

dataPath_train = 'c:\\Users\\YJKIM_PC\\pytorch_study\\sesac\\Vehicles_tvt\\train'
dataPath_valid = 'c:\\Users\\YJKIM_PC\\pytorch_study\\sesac\\Vehicles_tvt\\valid'
dataPath_test = 'c:\\Users\\YJKIM_PC\\pytorch_study\\sesac\\Vehicles_tvt\\test'

# for folder in dataFolder:
#     classFolder = os.path.join(dataPath, folder)
#     fileList = os.listdir(classFolder)
#     dataSize = len(fileList)
#     trainCount = int(dataSize * train_ratio)
#     validCount = int(dataSize * valid_ratio)
#     testCount = dataSize - trainCount - validCount
#     # print(dataSize)
#     indices = list(range(dataSize))
#     # print(indices)

#     trainList = random.sample(indices, trainCount)
#     remaining_indices = list(set(indices) - set(trainList))
#     # print(remaining_indices)

#     validList = random.sample(remaining_indices, validCount)

#     testList = list(set(remaining_indices) - set(validList))

#     trainData = [fileList[i] for i in trainList]
#     validData = [fileList[i] for i in validList]
#     testData = [fileList[i] for i in testList]

#     for data in trainData:
#         oldPath = os.path.join(classFolder, data)
#         newPath = os.path.join(dataPath_train, folder)
#         # print(oldPath, newPath)
#         # break
#         shutil.move(oldPath, newPath)
#     for data in validData:
#         oldPath = os.path.join(classFolder, data)
#         newPath = os.path.join(dataPath_valid, folder)
#         # print(oldPath, newPath)
#         # break
#         shutil.move(oldPath, newPath)
#     for data in testData:
#         oldPath = os.path.join(classFolder, data)
#         newPath = os.path.join(dataPath_test, folder)
#         # print(oldPath, newPath)
#         # break
#         shutil.move(oldPath, newPath)

In [8]:
for folder in dataFolder:
    print(folder, len(os.listdir(os.path.join(dataPath_train, folder))))
print('----------')
for folder in dataFolder:
    print(folder, len(os.listdir(os.path.join(dataPath_valid, folder))))
print('----------')
for folder in dataFolder:
    print(folder, len(os.listdir(os.path.join(dataPath_test, folder))))

Auto Rickshaws 640
Bikes 640
Cars 632
Motorcycles 640
Planes 640
Ships 640
Trains 640
----------
Auto Rickshaws 120
Bikes 120
Cars 118
Motorcycles 120
Planes 120
Ships 120
Trains 120
----------
Auto Rickshaws 40
Bikes 40
Cars 40
Motorcycles 40
Planes 40
Ships 40
Trains 40


In [15]:
import torch
import torchvision
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.models import resnet50

data_dir = 'c:\\Users\\YJKIM_PC\\pytorch_study\\sesac\\Vehicles_tvt'

In [52]:
# 아래 코드로 모든 이미지가 140, 140인지는 알 수 없음.

import cv2

img_dir = os.path.join(dataPath_train, dataFolder[0])
img_test = os.listdir(img_dir)

img = cv2.imread(os.path.join(img_dir, img_test[0]))
img.shape

(140, 140, 3)

In [56]:
from torchvision import transforms

# 2는 나중에 실험해보기(연산량 늘 것 같음)
org_size1 = (224, 224)
org_size2 = (256, 256)

transform = transforms.Compose([
    transforms.Resize(org_size1),
    transforms.ToTensor()
])

In [64]:
from torchvision.datasets import ImageFolder

class ImageFolderWithFilenames(ImageFolder):
    def __getitem__(self, index):
        original_tuple = super(ImageFolderWithFilenames, self).__getitem__(index)

        path, _ = self.samples[index]

        return original_tuple + (path, )

In [65]:
train_data = ImageFolder(root=f'{data_dir}\\train', transform=transform)
valid_data = ImageFolder(root=f'{data_dir}\\valid', transform=transform)
test_data = ImageFolderWithFilenames(root=f'{data_dir}\\test', transform=transform)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=16, shuffle=True)
test_loader = DataLoader(test_data, batch_size=1, shuffle=False)

In [75]:
# 클래스 목록 출력
print(train_data.classes)

# 클래스와 인덱스의 매핑 출력
print(train_data.class_to_idx)

['Auto Rickshaws', 'Bikes', 'Cars', 'Motorcycles', 'Planes', 'Ships', 'Trains']
{'Auto Rickshaws': 0, 'Bikes': 1, 'Cars': 2, 'Motorcycles': 3, 'Planes': 4, 'Ships': 5, 'Trains': 6}


In [13]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [16]:
model = resnet50(pretrained=True).to(device)

model

c:\Users\YJKIM_PC\anaconda3\envs\pt\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\YJKIM_PC\anaconda3\envs\pt\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [17]:
%pip install pytorch-ignite

Note: you may need to restart the kernel to use updated packages.


In [30]:
path = os.path.join(os.getcwd(), 'vehicle_model')

if not os.path.exists(path):
    os.makedirs(path)

In [32]:
class EarlyStopping:
    def __init__(self, patience=7, verbose=False, delta=0, path='.\\vehicle_model\\'):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = float('inf')
        self.delta = delta
        self.path = path

    def __call__(self, val_loss, model):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}). Saving model ...')
            torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

early_stopping = EarlyStopping(patience=8, verbose=True, path = path + 'resnet50_best.pth')

In [73]:
model = resnet50(pretrained=True).to(device)

num_features = model.fc.in_features

num_classes = 7

model.fc = torch.nn.Sequential(
    torch.nn.Linear(num_features, 256),
    torch.nn.ReLU(),
    torch.nn.BatchNorm1d(256),
    torch.nn.Dropout(0.5),
    torch.nn.Linear(256, num_classes)
).to(device)

model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [34]:
import time
from tqdm import tqdm
import matplotlib.pyplot as plt

def fit(model, criterion, optimizer, epochs, train_loader, valid_loader, resume=False):

    if resume == False:
        train_losses = []
        train_accuracies = []
        valid_losses = []
        valid_accuracies = []
    
    for epoch in range(epochs):
        start = time.time()

        model.train()

        train_loss = 0
        train_acc = 0
        train_correct = 0

        for train_x, train_y in tqdm(train_loader):
            train_x, train_y = train_x.to(device), train_y.to(device)
            outputs = model(train_x)

            loss = criterion(outputs, train_y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            train_correct += (predicted == train_y).sum().item()

        valid_loss = 0
        valid_acc = 0
        valid_correct = 0

        for valid_x, valid_y in valid_loader:
            with torch.no_grad():
                model.eval()
                valid_x, valid_y = valid_x.to(device), valid_y.to(device)
                outputs = model(valid_x)

                loss = criterion(outputs, valid_y)

                valid_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                valid_correct += (predicted == valid_y).sum().item()

        train_acc = train_correct / len(train_loader.dataset)
        valid_acc = valid_correct / len(valid_loader.dataset)

        print(f'{time.time() - start:.3f}sec : [Epoch {epoch+1}/{epochs}] -> train loss : {train_loss/len(train_loader):.4f}, train_acc : {train_acc*100:.3f}% / valid loss : {valid_loss / len(valid_loader):.4f}, valid_acc : {valid_acc*100:.3f}%')

        train_losses.append(train_loss / len(train_loader))
        train_accuracies.append(train_acc)
        valid_losses.append(valid_loss / len(valid_loader))
        valid_accuracies.append(valid_acc)

        scheduler.step(valid_loss)

        now_lr = optimizer.param_groups[0]['lr']
        print(f'learning_rate {epoch+1} : {now_lr:.8f}')

        early_stopping(valid_loss, model)

        if early_stopping.early_stop:
            print('Early stopping triggered.')
            break
        print('-' * 100)

    plt.plot(train_losses, label='loss')
    plt.plot(train_accuracies, label='accuracy')
    plt.legend()
    plt.title('train loss and accuracy')
    plt.show()

    plt.plot(valid_losses, label='loss')
    plt.plot(valid_accuracies, label='accuracy')
    plt.legend()
    plt.title('valid loss and accuracy')
    plt.show()

In [74]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

lr = 0.001
epochs = 30

criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)

fit(model, criterion, optimizer, epochs, train_loader, valid_loader, resume=False)

 49%|████▊     | 34/70 [05:05<05:23,  9.00s/it]


KeyboardInterrupt: 

In [67]:
import matplotlib.pyplot as plt
from PIL import Image

def evaluate_and_visualize(model, test_loader, criterion):
    test_loss = 0
    test_correct = 0
    test_total = 0

    incorrect_labels = []
    incorrect_preds = []
    incorrect_filenames = []

    model.eval()
    with torch.no_grad():
        for test_x, test_y, filenames in tqdm(test_loader):
            test_x, test_y = test_x.to(device), test_y.to(device)
            outputs = model(test_x)

            loss = criterion(outputs, test_y)
            test_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)

            test_correct += (predicted == test_y).sum().item()
            test_total += test_y.size(0)

            if predicted != test_y:
                incorrect_labels.append(test_y.cpu().numpy())
                incorrect_preds.append(predicted.cpu().numpy())
                incorrect_filenames.append(filenames[0])

    test_accuracy = test_correct / test_total
    print(f'test_correct : {test_correct}, test_total : {test_total}')
    print(f'Test Loss : {test_loss / len(test_loader):.4f}, Test Accuracy : {test_accuracy * 100:.2f}%')

    print(f'틀린 개수: {len(incorrect_filenames)}')
    
    # label이 없어서 아직 시각화를 못 하겠음

    # fig = plt.figure(figsize=(12,12))
    # for i in range(min(16, len(incorrect_filenames))):
    #     ax = fig.add_subplot(4, 4, i + 1)
    #     image = Image.open(incorrect_filenames[i])
    #     ax.imshow(image)

In [68]:
evaluate_and_visualize(model, test_loader, criterion)

100%|██████████| 280/280 [00:39<00:00,  7.13it/s]

test_correct : 252, test_total : 280
Test Loss : 0.7734, Test Accuracy : 90.00%
틀린 개수: 28
